# Sales Copilot - Multi-Market Sales & Product Finder (RAG with OpenAI)

## Project Goal

The goal of the **Sales Copilot — Multi-Market Sales & Product Finder** project is to provide a scalable, AI-powered tool that enables corporate sales teams to quickly identify the best matching products for international customers based on their technical requirements, budget constraints, and market availability. The system leverages a **Retrieval-Augmented Generation (RAG)** architecture that combines semantic search, keyword-based retrieval, reranking models, and an LLM to deliver precise, context-grounded answers. By processing heterogeneous data formats such as PDF specifications, Excel price lists, CSV inventory reports, and promotional catalogs, the Sales Copilot can unify knowledge from multiple sources and markets into a single intelligent search and answer interface. For this demonstration, all data used in the project was generated synthetically, simulating realistic product, pricing, inventory, and promotion datasets.



In practice, this solution aims to reduce the time spent on manual product lookup, improve sales accuracy, and ensure that recommendations are both technically valid and commercially relevant. Sales teams can query the system in natural language (e.g., “Find a silent HVAC unit for Germany with airflow above 700 m³/h and IP54 protection”), and the tool will retrieve relevant documents, rank them by relevance, and produce a clear, evidence-backed recommendation with price, availability, and promotion details for the specific market. By integrating reranking and LLM-based grounded answers, the Sales Copilot provides results that are both highly relevant and trustworthy, enabling faster decision-making and improving customer satisfaction across international markets.



## Setup

In [1]:
# Mounting to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "your-path-here"

In [3]:
# Install required libraries
%%capture

!pip -q install sentence-transformers faiss-cpu rank_bm25 pdfplumber reportlab rapidfuzz tabulate openai

import os, re, glob, random, numpy as np, pandas as pd
from datetime import datetime
from collections import defaultdict
import textwrap
import os

from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
import faiss
import pdfplumber
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4

try:
    from openai import OpenAI
    OPENAI_OK = True
except Exception:
    OPENAI_OK = False

SEED=42
random.seed(SEED); np.random.seed(SEED)
print("Environment ready. OPENAI_OK =", OPENAI_OK)

In [ ]:
# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-oppenai-api-key-here"

## Generate Synthetic Data (PDF/Text + CSV/Excel)

In [5]:
# We create a small catalog of HVAC units with attributes + country-specific tables.
DATA_DIR = "data_synthetic"
os.makedirs(DATA_DIR, exist_ok=True)

def make_spec_pdf(path, title, lines):
    c = canvas.Canvas(path, pagesize=A4)
    width, height = A4
    c.setFont("Helvetica-Bold", 14)
    c.drawString(40, height-50, title)
    c.setFont("Helvetica", 11)
    y = height-90
    for ln in lines:
        c.drawString(40, y, ln)
        y -= 16
        if y < 80:
            c.showPage()
            y = height-50
    c.save()

countries = ["DE","PL","ES"]
currencies = {"DE":"EUR","PL":"PLN","ES":"EUR"}

# Create synthetic product catalog
products = []
for i in range(1, 16):
    sku = f"HVX-{i:03d}"
    airflow = random.choice([300,400,500,700,900,1200])
    noise = random.choice([38,42,45,48,52])
    ip = random.choice(["IP20","IP44","IP54"])
    voltage = random.choice(["230V","110V"])
    title = f"{sku} Quiet HVAC Unit"
    products.append(dict(sku=sku, airflow=airflow, noise=noise, ip=ip, voltage=voltage, title=title))

# Save specs as CSV text and individual PDFs
spec_rows = []
for p in products:
    spec_text = f"""# {p['title']}
SKU: {p['sku']}
Airflow: {p['airflow']} m3/h
Noise: {p['noise']} dB(A)
Protection: {p['ip']}
Voltage: {p['voltage']}
Applications: Quiet cooling for commercial buildings.
"""
    spec_rows.append(dict(sku=p["sku"], text=spec_text, country="GLOBAL", section="spec"))
    make_spec_pdf(os.path.join(DATA_DIR, f"spec_{p['sku']}.pdf"), p["title"], [
        f"SKU: {p['sku']}",
        f"Airflow: {p['airflow']} m3/h",
        f"Noise level: {p['noise']} dB(A)",
        f"Protection rating: {p['ip']}",
        f"Voltage: {p['voltage']}",
        "Applications: Quiet cooling for commercial buildings."
    ])
pd.DataFrame(spec_rows).to_csv(os.path.join(DATA_DIR,"spec_texts.csv"), index=False)

# Country price lists
price_rows = []
for c in countries:
    for p in products:
        base = 1000 + p["airflow"]*0.8 + (55 - p["noise"])*10 + (10 if p["ip"]=="IP54" else 0)
        adj = {"DE":1.0,"PL":0.95,"ES":1.02}[c]
        price = round(base*adj, 2)
        price_rows.append(dict(sku=p["sku"], country=c, currency=currencies[c],
                               price=price, valid_from="2025-08-01", valid_to="2025-09-30"))
prices = pd.DataFrame(price_rows)
prices.to_csv(os.path.join(DATA_DIR,"price_list.csv"), index=False)
prices.to_excel(os.path.join(DATA_DIR,"price_list.xlsx"), index=False)

# Inventory snapshots
inv_rows = []
for c in countries:
    for p in products:
        available = random.choice([0,3,5,10,20])
        lead_weeks = random.choice([1,2,4,6,8])
        inv_rows.append(dict(sku=p["sku"], country=c, available=available, lead_weeks=lead_weeks, snapshot="2025-08-10"))
inventory = pd.DataFrame(inv_rows)
inventory.to_csv(os.path.join(DATA_DIR,"inventory.csv"), index=False)

# Promotions
promo_rows = []
for c in countries:
    for p in random.sample(products, 5):
        promo_rows.append(dict(sku=p["sku"], country=c, description=f"Back-to-work discount in {c}",
                               discount_pct=random.choice([5,10,12,15]), valid_to="2025-09-15"))
promos = pd.DataFrame(promo_rows)
promos.to_csv(os.path.join(DATA_DIR,"promos.csv"), index=False)

# A tidy product dataframe (for constraint checks later)
prod_df = pd.DataFrame(products)
print("Data ready:", len(prod_df), "products")
prod_df.head()

Data ready: 15 products


,sku,airflow,noise,ip,voltage,title
0,HVX-001,1200,38,IP20,110V,HVX-001 Quiet HVAC Unit
1,HVX-002,400,42,IP20,230V,HVX-002 Quiet HVAC Unit
2,HVX-003,1200,52,IP20,110V,HVX-003 Quiet HVAC Unit
3,HVX-004,300,38,IP20,230V,HVX-004 Quiet HVAC Unit
4,HVX-005,400,52,IP54,230V,HVX-005 Quiet HVAC Unit


## Parse PDFs, Build Corpus (Chunks + Metadata)

In [6]:
def read_pdf_text(path):
    txt=[]
    with pdfplumber.open(path) as pdf:
        for pg in pdf.pages:
            txt.append(pg.extract_text() or "")
    return "\n".join(txt)

spec_texts = pd.read_csv(os.path.join(DATA_DIR,"spec_texts.csv"))
pdf_files = glob.glob(os.path.join(DATA_DIR, "spec_*.pdf"))
pdf_rows=[]
for f in pdf_files:
    sku = os.path.basename(f).split("_")[1].split(".")[0]
    pdf_rows.append(dict(sku=sku, text=read_pdf_text(f), country="GLOBAL", section="spec"))
spec_pdfs = pd.DataFrame(pdf_rows)

# Build one row per “chunk” (here: per record) to feed the index
corpus_rows=[]
for df in [spec_texts, spec_pdfs]:
    for _, r in df.iterrows():
        corpus_rows.append(dict(
            doc_id=f"spec:{r['sku']}",
            chunk_id=f"{r['sku']}:spec",
            text=r["text"],
            lang="en",
            sku=r["sku"],
            country="GLOBAL",
            section="spec"
        ))

for _, r in prices.iterrows():
    corpus_rows.append(dict(
        doc_id=f"price:{r['sku']}:{r['country']}",
        chunk_id=f"{r['sku']}:{r['country']}:price",
        text=f"Price for {r['sku']} in {r['country']}: {r['price']} {r['currency']} valid {r['valid_from']} to {r['valid_to']}.",
        lang="en",
        sku=r["sku"], country=r["country"], section="pricing",
        currency=r["currency"], valid_from=r["valid_from"], valid_to=r["valid_to"]
    ))

for _, r in inventory.iterrows():
    corpus_rows.append(dict(
        doc_id=f"inv:{r['sku']}:{r['country']}",
        chunk_id=f"{r['sku']}:{r['country']}:inv",
        text=f"Inventory for {r['sku']} in {r['country']}: available {r['available']}, lead time {r['lead_weeks']} weeks (snapshot {r['snapshot']}).",
        lang="en",
        sku=r["sku"], country=r["country"], section="inventory"
    ))

for _, r in promos.iterrows():
    corpus_rows.append(dict(
        doc_id=f"promo:{r['sku']}:{r['country']}",
        chunk_id=f"{r['sku']}:{r['country']}:promo",
        text=f"Promotion {r['discount_pct']}% for {r['sku']} in {r['country']} until {r['valid_to']}. {r['description']}",
        lang="en",
        sku=r["sku"], country=r["country"], section="promo"
    ))

corpus = pd.DataFrame(corpus_rows)
print("Corpus size:", len(corpus))
corpus.head()

Corpus size: 135


,doc_id,chunk_id,text,lang,sku,country,section,currency,valid_from,valid_to
0,spec:HVX-001,HVX-001:spec,# HVX-001 Quiet HVAC Unit\nSKU: HVX-001\nAirfl...,en,HVX-001,GLOBAL,spec,NaN,NaN,NaN
1,spec:HVX-002,HVX-002:spec,# HVX-002 Quiet HVAC Unit\nSKU: HVX-002\nAirfl...,en,HVX-002,GLOBAL,spec,NaN,NaN,NaN
2,spec:HVX-003,HVX-003:spec,# HVX-003 Quiet HVAC Unit\nSKU: HVX-003\nAirfl...,en,HVX-003,GLOBAL,spec,NaN,NaN,NaN
3,spec:HVX-004,HVX-004:spec,# HVX-004 Quiet HVAC Unit\nSKU: HVX-004\nAirfl...,en,HVX-004,GLOBAL,spec,NaN,NaN,NaN
4,spec:HVX-005,HVX-005:spec,# HVX-005 Quiet HVAC Unit\nSKU: HVX-005\nAirfl...,en,HVX-005,GLOBAL,spec,NaN,NaN,NaN


## Build BM25 (keywords) + FAISS (semantic embeddings)

In [7]:
# BM25: keyword retrieval (great for exact SKUs/terms)
tokenized = [t.lower().split() for t in corpus['text'].tolist()]
bm25 = BM25Okapi(tokenized)

# Dense embeddings (multilingual-e5) for semantic search
embed_model = SentenceTransformer("intfloat/multilingual-e5-base")

def e5_embed(texts, mode="passage"):
    prefix = "passage: " if mode=="passage" else "query: "
    return embed_model.encode([prefix + t for t in texts], normalize_embeddings=True, show_progress_bar=True)

embs = e5_embed(corpus["text"].tolist(), mode="passage").astype("float32")

# FAISS HNSW index (fast ANN search)
index = faiss.IndexHNSWFlat(embs.shape[1], 32)
index.hnsw.efConstruction = 80
index.add(embs)

# Cross-encoder for reranking (precision boost)
cross = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

print("FAISS size:", index.ntotal)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

FAISS size: 135


## Retrieval: RRF fusion + cross-encoder rerank

In [8]:
def retrieve(query:str, country:str=None, k_dense=200, k_bm25=200):
    # Dense candidate IDs
    q_emb = e5_embed([query], mode="query").astype("float32")
    D, I = index.search(q_emb, k_dense)
    dense_ids = I[0].tolist()

    # BM25 candidate IDs
    bm_scores = bm25.get_scores(query.lower().split())
    bm_top = np.argsort(bm_scores)[-k_bm25:][::-1].tolist()

    # Reciprocal Rank Fusion (simple, robust)
    rank_dense = {idx:i+1 for i,idx in enumerate(dense_ids)}
    rank_bm    = {idx:i+1 for i,idx in enumerate(bm_top)}
    all_ids = list(set(dense_ids) | set(bm_top))
    fused = []
    for idx in all_ids:
        rd = rank_dense.get(idx, 10**6)
        rb = rank_bm.get(idx, 10**6)
        score = 1/(60+rd) + 1/(60+rb)
        fused.append((idx, score))
    fused.sort(key=lambda x: x[1], reverse=True)

    # Filter by country (GLOBAL always allowed)
    filtered = []
    for idx, sc in fused:
        row = corpus.iloc[idx]
        if country and row.get("country","GLOBAL") not in (country, "GLOBAL"):
            continue
        filtered.append((idx, sc))
    return filtered

def rerank(query:str, fused_ids_scores, final_k=10, top_for_rerank=50):
    topk = fused_ids_scores[:top_for_rerank]
    if not topk:
        return corpus.head(0)
    texts = [corpus.iloc[i]["text"] for i,_ in topk]
    pairs = [(query, t) for t in texts]
    scores = cross.predict(pairs).tolist()
    reranked = sorted([(topk[i][0], scores[i]) for i in range(len(topk))], key=lambda x: x[1], reverse=True)[:final_k]
    df = corpus.iloc[[i for i,_ in reranked]].copy()
    df["score"] = [s for _,s in reranked]
    return df

def search(query, country=None, final_k=10):
    fused = retrieve(query, country=country)
    return rerank(query, fused, final_k=final_k, top_for_rerank=50)

## Constraint parsing & candidate helpers

In [9]:
# Parse constraints (airflow/IP/voltage) from natural-language queries
def parse_constraints(query: str):
    q = query.lower()
    airflow, airflow_op = None, None
    m = re.search(r'(?:>=|≥|=>|at least|minimum|min\.?)?\s*(\d{2,5})\s*(?:m3/h|m³/h)', q)
    if m:
        airflow = int(m.group(1))
        airflow_op = '>=' if any(tok in q for tok in ['>=','≥','=>','at least','minimum','min']) else '>='

    ip_match = re.search(r'\bip\s*([0-9]{2})\b', q)
    ip = f"IP{ip_match.group(1)}" if ip_match else None

    volt_match = re.search(r'\b(110|230)\s*v\b', q)
    voltage = f"{volt_match.group(1)}V" if volt_match else None

    return {"airflow": airflow, "airflow_op": airflow_op, "ip": ip, "voltage": voltage}

# Join airflow/ip/voltage onto any result DF (based on SKU)
def _attach_specs(df):
    if df is None or df.empty:
        return df
    df2 = df.dropna(subset=["sku"]).copy()
    return df2.merge(prod_df[["sku","airflow","ip","voltage"]], on="sku", how="left")

def meets_constraints(row, cons):
    ok = True
    # airflow
    if cons.get("airflow") is not None and not pd.isna(row.get("airflow")):
        if cons.get("airflow_op", ">=") == ">=":
            ok &= row["airflow"] >= cons["airflow"]
        else:
            ok &= row["airflow"] == cons["airflow"]
    # ip
    if cons.get("ip"):
        ok &= (str(row.get("ip","")).upper() == cons["ip"].upper())
    # voltage
    if cons.get("voltage"):
        ok &= (str(row.get("voltage","")).upper() == cons["voltage"].upper())
    return bool(ok)

def filter_candidates_by_constraints(results_df, cons, top_n=10):
    df = _attach_specs(results_df)
    if df is None or df.empty:
        return df
    mask = df.apply(lambda r: meets_constraints(r, cons), axis=1)
    out = df[mask].drop_duplicates(subset=["sku"])
    return out.head(top_n)

def find_closest_candidates(results_df, cons, country, max_alts=3):
    """Rank near-misses by #constraints met (airflow with 90% tolerance, ip match, voltage match)."""
    df = _attach_specs(results_df)
    if df is None or df.empty:
        return df.head(0)

    scored = []
    req_af = cons.get("airflow")
    for _, r in df.drop_duplicates("sku").iterrows():
        score = 0.0
        if req_af is not None and not pd.isna(r["airflow"]):
            if r["airflow"] >= req_af: score += 1.0
            elif r["airflow"] >= 0.9 * req_af: score += 0.5
        if cons.get("ip") and isinstance(r["ip"], str) and r["ip"].upper() == cons["ip"].upper():
            score += 1.0
        if cons.get("voltage") and isinstance(r["voltage"], str) and r["voltage"].upper() == cons["voltage"].upper():
            score += 1.0
        if score > 0: scored.append((r["sku"], score))

    if not scored: return df.head(0)
    scored_df = pd.DataFrame(scored, columns=["sku","score"]).sort_values("score", ascending=False)
    merged = scored_df.merge(df.drop_duplicates("sku"), on="sku", how="left")
    return merged.head(max_alts)

## Formatting helpers for answers (bullets & facts blocks)

In [10]:
def format_match_bullets(sku, country):
    """Neat bullets with citations: SKU + (optional) price, availability, promo."""
    lines = [f"- Suggested SKU: **{sku}**  [spec:{sku}]"]

    pr = prices[(prices.sku==sku) & (prices.country==country)]
    if not pr.empty:
        pr0 = pr.iloc[0]
        lines.append(f"- Price: **{pr0.price} {pr0.currency}** (valid {pr0.valid_from}–{pr0.valid_to})  "
                     f"[price:{sku}:{pr0.country}]")
    iv = inventory[(inventory.sku==sku) & (inventory.country==country)]
    if not iv.empty:
        iv0 = iv.iloc[0]
        lines.append(f"- Availability: **{iv0.available} units**, lead **{iv0.lead_weeks} weeks**  "
                     f"[inv:{sku}:{iv0.country}]")
    pm = promos[(promos.sku==sku) & (promos.country==country)]
    if not pm.empty:
        pm0 = pm.iloc[0]
        lines.append(f"- Promotion: **{pm0.discount_pct}%** until **{pm0.valid_to}**  "
                     f"[promo:{sku}:{pm0.country}]")
    return "\n".join(lines)

def build_candidate_facts_block(df, country, max_cands=3):
    """Compact facts block (with citations) used to ground the LLM."""
    if df is None or df.empty:
        return "None"
    rows = []
    for _, r in df.head(max_cands).iterrows():
        sku = r.sku
        parts = [f"- {sku}: airflow {r.airflow} m3/h, {r.ip}, {r.voltage}  [spec:{sku}]"]
        pr = prices[(prices.sku==sku) & (prices.country==country)]
        if not pr.empty:
            pr0 = pr.iloc[0]
            parts.append(f"price {pr0.price} {pr0.currency} (valid {pr0.valid_from}–{pr0.valid_to}) [price:{sku}:{country}]")
        iv = inventory[(inventory.sku==sku) & (inventory.country==country)]
        if not iv.empty:
            iv0 = iv.iloc[0]
            parts.append(f"availability {iv0.available} (lead {iv0.lead_weeks} w) [inv:{sku}:{country}]")
        pm = promos[(promos.sku==sku) & (promos.country==country)]
        if not pm.empty:
            pm0 = pm.iloc[0]
            parts.append(f"promo {pm0.discount_pct}% until {pm0.valid_to} [promo:{sku}:{country}]")
        rows.append(" ; ".join(parts))
    return "\n".join(rows)

## Deterministic fallback answer (no LLM used)

In [11]:
def compose_answer_fallback(query, country, results_df):
    cons = parse_constraints(query)
    exact = filter_candidates_by_constraints(results_df, cons, top_n=5)

    # No exact match: show helpful alternatives
    if exact is None or exact.empty:
        closest = find_closest_candidates(results_df, cons, country, max_alts=3)
        intro = "I couldn’t find a product that meets all of your constraints in the current results.\n"
        if closest is not None and not closest.empty:
            alt_intro = "\nHere are some close alternatives you might consider:\n"
            alt_lines = [format_match_bullets(r.sku, country) for _, r in closest.iterrows()]
            alt_block = "\n\n".join(alt_lines)
        else:
            alt_intro, alt_block = "", ""
        steps = (
            "\nYou could try:\n"
            "- Loosening the airflow threshold a bit\n"
            "- Allowing a different voltage if acceptable\n"
            "- Checking back later (inventory & promos change)\n"
        )
        text = intro + alt_intro + alt_block + steps
        return "\n".join(textwrap.fill(line, width=88) if line and not line.startswith("- ") else line
                         for line in text.split("\n"))

    # Found match: friendly intro + bullets + light outro
    best_sku = exact.iloc[0]["sku"]
    intro = ("Great news — I found an option that matches your requirements. "
             "Here are the details:\n")
    bullets = format_match_bullets(best_sku, country)
    outro = ("\n\nIf you’d like, I can compare this SKU with a couple of close alternatives "
             "or check other countries.")
    wrapped = intro + bullets + outro
    return "\n".join(textwrap.fill(line, width=88) if line and not line.startswith("- ")
                     else line for line in wrapped.split("\n"))

## LLM (OpenAI) grounded answer

In [12]:
import textwrap, os, re

def compose_answer_openai_strict(
    query,
    country,
    results_df,
    model="gpt-4o-mini",
    temperature=0.2,
    max_tokens=500,          # ↑ give the model more room
    max_evidence=6,          # ↓ keep context concise
    max_exact=3,
    max_close=3,
    _retry=False             # internal: one-shot retry flag
):
    try:
        from openai import OpenAI
        api_key = os.environ.get("OPENAI_API_KEY", "")
        if not api_key:
            return None
        client = OpenAI(api_key=api_key)
    except Exception:
        return None

    # ---- constraint parsing & candidate sets ----
    cons        = parse_constraints(query)
    exact_df    = filter_candidates_by_constraints(results_df, cons, top_n=max_exact)
    closest_df  = find_closest_candidates(results_df, cons, country, max_alts=max_close)

    def shortlist_lines(df):
        if df is None or df.empty: return []
        return [f"- {r.sku}: airflow {r.airflow} m3/h, {r.ip}, {r.voltage} [spec:{r.sku}]"
                for _, r in df.iterrows()]

    shortlist          = "\n".join(shortlist_lines(exact_df)) or "None"
    facts_block_exact  = build_candidate_facts_block(exact_df, country, max_cands=max_exact)
    facts_block_close  = build_candidate_facts_block(closest_df, country, max_cands=max_close)

    evidence = "\n".join(
        f"[{row['doc_id']}] {row['text']}"
        for _, row in results_df.head(max_evidence).iterrows()
    )

    system = (
        "You are a helpful, friendly sales assistant. Be concise and approachable.\n"
        "Base your answer ONLY on the provided shortlist, facts, closest options, and context.\n"
        "Formatting rules:\n"
        "- Use short paragraphs; avoid long lines.\n"
        "- If a match exists, start with a friendly confirmation, then bullet points:\n"
        "  - Suggested SKU, Price (with validity), Availability, Promotions (cite as [doc_id]).\n"
        "- If no match exists, say so briefly, then list 'Close Alternatives' as bullet points if any.\n"
        "- If no close alternatives exist, give 2–3 helpful next steps.\n"
        "- Always include citations like [doc_id]."
    )

    user = f"""Query: {query} (country={country})

Constraints parsed: {cons}

Candidate shortlist (meets constraints):
{shortlist}

Candidate facts (with citations):
{facts_block_exact}

Closest alternatives (with citations):
{facts_block_close}

Context (evidence):
{evidence}

Answer:"""

    try:
        resp = client.chat.completions.create(
            model=model,
            messages=[{"role":"system","content":system},
                      {"role":"user","content":user}],
            temperature=temperature,
            max_tokens=max_tokens
        )
        raw_text = resp.choices[0].message.content or ""

        # ---- simple truncation heuristics ----
        def looks_truncated(txt: str) -> bool:
            # dangling bullet or started listing close alternatives but ended abruptly
            lines = [ln.rstrip() for ln in txt.split("\n")]
            if any(ln.strip().startswith(("- **", "-**", "- •", "•")) and len(ln.strip()) < 6 for ln in lines[-3:]):
                return True
            # if we expected close alts and got fewer than max_close entries
            if (closest_df is not None and not closest_df.empty):
                # count bullets that start with "- **HVX"
                count = sum(bool(re.match(r"^\s*-\s*\*\*HVX-", ln)) for ln in lines)
                if count < min(len(closest_df), max_close):
                    # Allow if the model chose to summarize; otherwise treat as truncation if it ended on a bullet
                    if lines and lines[-1].lstrip().startswith("- "):
                        return True
            return False

        if looks_truncated(raw_text) and not _retry and max_tokens < 900:
            # Auto-retry once with a larger token budget
            return compose_answer_openai_strict(
                query, country, results_df, model=model, temperature=temperature,
                max_tokens=min(900, max_tokens * 2), max_evidence=max_evidence,
                max_exact=max_exact, max_close=max_close, _retry=True
            )

        # Wrap long lines for notebook readability (keep bullets intact)
        wrapped = "\n".join(
            textwrap.fill(line, width=88) if (line and not line.startswith("- "))
            else line
            for line in raw_text.split("\n")
        )
        return wrapped

    except Exception:
        return None

## Orchestration helpers: show results + print both answers

In [13]:
def ask(query, country="DE", use_llm=False, top_n=10):
    """Basic runner: retrieval → results table → one answer (LLM OR fallback)."""
    results = search(query, country=country, final_k=top_n)
    display(results[["doc_id","section","sku","country","text"]])
    print("\n---\nAnswer:")
    if use_llm:
        print(compose_answer_openai_strict(query, country, results) or "(LLM unavailable)")
    else:
        print(compose_answer_fallback(query, country, results))

def ask_both(query, country="DE", top_n=10):
    """Runner that prints BOTH answers: fallback first, then LLM (if available)."""
    results = search(query, country=country, final_k=top_n)
    display(results[["doc_id","section","sku","country","text"]])
    print("\n--- Answer (Deterministic Fallback) ---")
    print(compose_answer_fallback(query, country, results))
    print("\n--- Answer (OpenAI LLM, grounded) ---")
    print(compose_answer_openai_strict(query, country, results) or "(LLM unavailable)")

In [14]:
# Demo: asking question

ask_both("Silent HVAC unit >= 700 m3/h, IP54, 230V for Germany", country="DE", top_n=8)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,doc_id,section,sku,country,text
12,spec:HVX-013,spec,HVX-013,GLOBAL,# HVX-013 Quiet HVAC Unit\nSKU: HVX-013\nAirfl...
28,spec:HVX-013,spec,HVX-013,GLOBAL,HVX-013 Quiet HVAC Unit\nSKU: HVX-013\nAirflow...
11,spec:HVX-012,spec,HVX-012,GLOBAL,# HVX-012 Quiet HVAC Unit\nSKU: HVX-012\nAirfl...
4,spec:HVX-005,spec,HVX-005,GLOBAL,# HVX-005 Quiet HVAC Unit\nSKU: HVX-005\nAirfl...
15,spec:HVX-005,spec,HVX-005,GLOBAL,HVX-005 Quiet HVAC Unit\nSKU: HVX-005\nAirflow...
29,spec:HVX-012,spec,HVX-012,GLOBAL,HVX-012 Quiet HVAC Unit\nSKU: HVX-012\nAirflow...
7,spec:HVX-008,spec,HVX-008,GLOBAL,# HVX-008 Quiet HVAC Unit\nSKU: HVX-008\nAirfl...
6,spec:HVX-007,spec,HVX-007,GLOBAL,# HVX-007 Quiet HVAC Unit\nSKU: HVX-007\nAirfl...



--- Answer (Deterministic Fallback) ---
I couldn’t find a product that meets all of your constraints in the current results.

Here are some close alternatives you might consider:
- Suggested SKU: **HVX-013**  [spec:HVX-013]
- Price: **1740.0 EUR** (valid 2025-08-01–2025-09-30)  [price:HVX-013:DE]
- Availability: **5 units**, lead **1 weeks**  [inv:HVX-013:DE]
- Promotion: **15%** until **2025-09-15**  [promo:HVX-013:DE]

- Suggested SKU: **HVX-012**  [spec:HVX-012]
- Price: **1320.0 EUR** (valid 2025-08-01–2025-09-30)  [price:HVX-012:DE]
- Availability: **20 units**, lead **2 weeks**  [inv:HVX-012:DE]

- Suggested SKU: **HVX-005**  [spec:HVX-005]
- Price: **1360.0 EUR** (valid 2025-08-01–2025-09-30)  [price:HVX-005:DE]
- Availability: **3 units**, lead **4 weeks**  [inv:HVX-005:DE]
You could try:
- Loosening the airflow threshold a bit
- Allowing a different voltage if acceptable
- Checking back later (inventory & promos change)


--- Answer (OpenAI LLM, grounded) ---
Unfortunately, t